In [ ]:
#@title 1. Setup & Caching#@markdown This cell handles all installations, downloads, and caching.#@markdown It will connect to your Google Drive and store everything in `/content/drive/MyDrive/AI_Art_Station`.#@markdown Subsequent runs will be much faster.import osimport timefrom google.colab import driveprint("🚀 Starting setup...")start_time = time.time()# --- 1. Mount Google Drive ---print("\n🔄 [1/7] Mounting Google Drive...")drive.mount('/content/drive')print("✅ Google Drive mounted successfully!")# --- 2. Define Paths & Create Directories ---print("\n🔄 [2/7] Setting up directory structure...")GDRIVE_ROOT = "/content/drive/MyDrive/AI_Art_Station"working_dir = GDRIVE_ROOTcomfyui_path = os.path.join(working_dir, 'ComfyUI')custom_nodes_path = os.path.join(comfyui_path, 'custom_nodes')models_path = os.path.join(comfyui_path, 'models')checkpoints_path = os.path.join(models_path, 'checkpoints')controlnet_path = os.path.join(models_path, 'controlnet')os.makedirs(checkpoints_path, exist_ok=True)os.makedirs(controlnet_path, exist_ok=True)os.makedirs(custom_nodes_path, exist_ok=True)print("✅ Directory structure is ready.")# --- 3. Install/Update ComfyUI ---print("\n🔄 [3/7] Checking and installing ComfyUI...")if os.path.exists(comfyui_path):    print("  -> ComfyUI already exists. Performing a quick update...")    %cd {comfyui_path}    !git pullelse:    print("  -> ComfyUI not found. Performing a fresh installation...")    %cd {working_dir}    !git clone https://github.com/comfyanonymous/ComfyUI.gitprint("✅ ComfyUI is ready.")# --- 4. Install/Update Custom Nodes ---print(f"\n🔄 Checking and installing custom node: ComfyUI-Manager...")node_path = os.path.join(custom_nodes_path, 'ComfyUI-Manager')if os.path.exists(node_path):    print(f"  -> ComfyUI-Manager already exists. Updating...")    %cd {node_path}    !git pullelse:    print(f"  -> Cloning ComfyUI-Manager...")    %cd {custom_nodes_path}    !git clone https://github.com/ltdrdata/ComfyUI-Managerprint(f"\n🔄 Checking and installing custom node: ComfyUI-AnimateDiff-Evolved...")node_path = os.path.join(custom_nodes_path, 'ComfyUI-AnimateDiff-Evolved')if os.path.exists(node_path):    print(f"  -> ComfyUI-AnimateDiff-Evolved already exists. Updating...")    %cd {node_path}    !git pullelse:    print(f"  -> Cloning ComfyUI-AnimateDiff-Evolved...")    %cd {custom_nodes_path}    !git clone https://github.com/Kosinkadink/ComfyUI-AnimateDiff-Evolvedprint("\n✅ All custom nodes are ready.")# --- 5. Install Dependencies ---print("\n🔄 [5/7] Installing Python dependencies...")%cd {comfyui_path}!pip install -r requirements.txt --extra-index-url https://download.pytorch.org/whl/cu121 -qprint("✅ Python dependencies installed.")# --- 6. Install Aria2c for Accelerated Downloads ---print("\n🔄 [6/7] Installing aria2c downloader...")!apt-get -y install -qq aria2print("✅ aria2c is ready.")# --- 7. Download Models ---print(f'\n🔄 Checking main model: stable-diffusion-v1-5...')model_file_path = os.path.join(checkpoints_path, 'v1-5-pruned-emaonly.ckpt')if not os.path.exists(model_file_path):    print(f'  -> Downloading v1-5-pruned-emaonly.ckpt...')    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M "https://huggingface.co/runwayml/stable-diffusion-v1-5/resolve/main/v1-5-pruned-emaonly.ckpt" -d "{checkpoints_path}" -o "v1-5-pruned-emaonly.ckpt"else:    print(f'  -> Model v1-5-pruned-emaonly.ckpt already exists.')print(f'\n🔄 Checking main model: dreamshaper-8...')model_file_path = os.path.join(checkpoints_path, 'dreamshaper-8.safetensors')if not os.path.exists(model_file_path):    print(f'  -> Downloading dreamshaper-8.safetensors...')    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M "https://huggingface.co/lykon/dreamshaper-8/resolve/main/dreamshaper-8.safetensors" -d "{checkpoints_path}" -o "dreamshaper-8.safetensors"else:    print(f'  -> Model dreamshaper-8.safetensors already exists.')print(f'\n🔄 Checking ControlNet model: canny...')model_file_path = os.path.join(controlnet_path, 'control_v11p_sd15_canny.pth')if not os.path.exists(model_file_path):    print(f'  -> Downloading control_v11p_sd15_canny.pth...')    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M "https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11p_sd15_canny.pth" -d "{controlnet_path}" -o "control_v11p_sd15_canny.pth"else:    print(f'  -> Model control_v11p_sd15_canny.pth already exists.')print(f'\n🔄 Checking ControlNet model: depth...')model_file_path = os.path.join(controlnet_path, 'control_v11f1p_sd15_depth.pth')if not os.path.exists(model_file_path):    print(f'  -> Downloading control_v11f1p_sd15_depth.pth...')    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M "https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11f1p_sd15_depth.pth" -d "{controlnet_path}" -o "control_v11f1p_sd15_depth.pth"else:    print(f'  -> Model control_v11f1p_sd15_depth.pth already exists.')print("\n✅ All models are checked and ready.")# --- Finalization ---end_time = time.time()print("\n" + "="*50)print(f"🎉 Setup complete! Total time: {end_time - start_time:.2f} seconds.")print("✅ You can now proceed to the next cell to launch the UI.")print("="*50)

In [ ]:
#@title 2. Launch ComfyUI#@markdown ### ⚙️ Hardware and Network Options#@markdown Select your hardware and preferred network tunnel.#@markdown - **`GPU`** is recommended for generating images.#@markdown - **`CPU`** is for workflow management without using GPU credits.#@markdown - **`Cloudflared`** is the recommended, stable, and unlimited tunnel.#@markdown ---hardware = "GPU"  #@param ["GPU", "CPU"]tunnel = "Cloudflared"  #@param ["Cloudflared", "Ngrok"]import osimport timeimport sys# --- 1. Define Paths ---GDRIVE_ROOT = "/content/drive/MyDrive/AI_Art_Station"comfyui_path = os.path.join(GDRIVE_ROOT, 'ComfyUI')# --- 2. Launch Tunnel Service ---if tunnel == 'Cloudflared':    print("🚀 Launching Cloudflared tunnel...")    !pkill cloudflared    !nohup cloudflared tunnel --url http://127.0.0.1:8188 > /content/cloudflared.log 2>&1 &    time.sleep(4) # Wait for log file to be created    print("🔗 Your Public URL:")    !grep -o 'https://[a-zA-Z0-9-]*\.trycloudflare.com' /content/cloudflared.log    print("✅ Cloudflared tunnel is active. Please use the link above.")else: # Ngrok    print("🚀 Launching Ngrok tunnel...")    !pip install pyngrok -q    from pyngrok import ngrok    ngrok.kill()    try:        ngrok_auth_token = os.environ.get('NGROK_AUTH_TOKEN', 'YOUR_NGROK_AUTHTOKEN_HERE')        ngrok.set_auth_token(ngrok_auth_token)        public_url = ngrok.connect(8188)        print(f"🔗 Your Public URL: {public_url}")        print("✅ Ngrok tunnel is active. (Note: Free tier has time limits)")    except Exception as e:        print(f"❌ Ngrok Error: {e}")        print("Please ensure your NGROK_AUTH_TOKEN is set correctly in Colab secrets.")# --- 3. Construct and Run Launch Command ---%cd {comfyui_path}launch_command = "python main.py --listen"if hardware == 'CPU':    launch_command += " --cpu-only" # Correct flag for CPU-only mode might varyelse:    launch_command += " --cuda-device 0" # Or other GPU-specific flagsprint(f"\n🔥 Launching ComfyUI with command: `{launch_command}`")print("🟢 Service is starting. This cell will continue to run.")!{launch_command}